In [ ]:
# !pip install indic-nlp-library
# !pip install fasttext
# !pip install inltk
import nltk
import pandas as pd
# import fasttext
import random
import numpy as np
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from indicnlp.tokenize import sentence_tokenize
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
url='https://secondhandbookimages.s3.ap-south-1.amazonaws.com/gujarati_train.jsonl'

train_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv_train.csv')
train_data.head()

,id,url,title,summary,text
0,international-52860599,https://www.bbc.com/gujarati/international-528...,લૉકડાઉન : પાકિસ્તાનમાં ફસાયેલા એ ગુજરાતીઓ જે ભ...,ગુજરાતના ગોધરાના ઇશાકભાઈ બોકડા અઢી મહિનાથી પાક...,પાકિસ્તાનમાં ફસાયેલા ભારતીય નાગરિકો ખેડૂત પરિવ...
1,india-44211023,https://www.bbc.com/gujarati/india-44211023,ગુજરાતમાં શા માટે વધી રહ્યા છે દલિતો પર અત્યાચ...,"11 જુલાઈ 2016, એટલે ઉના દલિતકાંડનો દિવસ.",આજથી ચાર વર્ષે પહેલાં આજના જ દિવસે ગીર-સોમનાથ ...
2,media-54889459,https://www.bbc.com/gujarati/media-54889459,પેટાચૂંટણીમાં હાર કૉંગ્રેસની કબર પર આખરી ખીલો ...,ગુજરાત વિધાનસભાની આઠ બેઠકો પર ત્રીજી નવેમ્બરે ...,સોમવારે હાથ ધરાયેલી મતગણતરીનાં પ્રારંભિક વલણોમ...
3,india-47014854,https://www.bbc.com/gujarati/india-47014854,જ્યારે પ્રણવ મુખર્જી બે વખત વડા પ્રધાન બનતા-બન...,પૂર્વ રાષ્ટ્રપતિ પ્રણવ મુખર્જીને ભારત સરકારે ભ...,ભારત સરકારે પૂર્વ રાષ્ટ્રપતિ પ્રણવ મુખર્જીનેભા...
4,india-49934787,https://www.bbc.com/gujarati/india-49934787,"ગુજરાતના સફેદ રણની રંગીન કહાણી, જે તમને આશ્ચર્...",ગુજરાતના અરબ સાગરથી 100 કિમી દૂર આવેલા બંજર રણ...,આ મેદાન કચ્છના રણના નામે ઓળખાય છે. કાચબાના આકા...


In [ ]:
# tokenize sentences
tokenized_sentences=[]
indic_string=train_data['text']
counter=0
for i in indic_string:
  print(counter, len(sentence_tokenize.sentence_split(i, lang='gu')))
  counter+=1
# indic_string="નરેન્દ્ર દામોદરદાસ મોદી (જન્મ: ૧૭ સપ્ટેમ્બર ૧૯૫૦)[૨] ભારતના ૧૪મા વડાપ્રધાન છે. તેઓ ગુજરાત રાજ્યના મુખ્ય મંત્રી પણ રહી ચૂક્યા છે અને ભારતીય જનતા પાર્ટી (ભાજપ)ના નેતા છે.નરેન્દ્ર દામોદરદાસ મોદીનો જન્મ એક મધ્યમ વર્ગના હિન્દુ કુટુંબમાં અમદાવાદથી ૧૧૨ કીમી અને મહેસાણાથી ૩૪ કીમી દૂર વડનગર ખાતે થયો હતો. તે દામોદરદાસ મૂલચંદ મોદી અને તેમના પત્ની હીરાબેન મોદીના છ સંતાન પૈકી ત્રીજુ સંતાન છે.[૩] તેમનું હુલામણું નામ એન.ડી. હતું. તે નાનપણથી જ રાષ્ટ્રીય સ્વયંસેવક સંઘના સભ્ય હતા. તેઓ ધાર્મિક હતા અને મહાદેવના મંદિરે જતા હતા. તેઓએ ૧૭ વર્ષની વયે ગૃહ ત્યાગ કર્યો હતો. તે હિમાલયમાં તેમજ રામકૃષ્ણ પરમહંસના બેલુર મઠમાં પણ રહી ચૂક્યા છે. ઈંદિરા ગાંધીએ લાદેલી કટોકટી વખતે તેઓ ભૂગર્ભમાં જતા રહ્યા હતા. આ કટોકટી વખતે જ તેમણે સંઘર્ષમાં ગુજરાત નામનું પુસ્તક લખ્યું હતું. તેમને કિશોરાવસ્થાથી રાજકારણમાં રસ હતો. તેમણે ગુજરાત યુનિવર્સિટીમાંથી રાજકીય વિજ્ઞાનમાં અનુસ્નાતકની પદવી પ્રાપ્ત કરી છે. ૧૯૯૮માં ભારતીય જનતા પાર્ટી (ભાજપ)ના નેતા લાલકૃષ્ણ અડવાણીએ ગુજરાત અને હિમાચલ પ્રદેશની ચૂંટણીમાં પ્રચાર માટે તેમની પસંદગી કરી હતી. ઇ.સ. ૨૦૦૧ના વર્ષમાં પૂર્વ મુખ્યમંત્રી કેશુભાઈ પટેલે રાજીનામું આપ્યા બાદ તેઓ ૭ ઓક્ટોબર ૨૦૦૧ના દિવસે મુખ્યમંત્રી બન્યા હતા. વર્ષ ૨૦૧૨ની ચૂંટણીમાં જંગી બહુમતીથી જીત્યા બાદ તેઓ ગુજરાત રાજ્યના સતત ચોથી વખત ચૂંટાઈ આવેલા એક માત્ર મુખ્યમંત્રી છે. મોદી ભારતની અંદર અને આંતરરાષ્ટ્રીય સ્તરે વિવાદમાં રહી ચૂક્યા છે"
# for i in sentence_tokenize.sentence_split(indic_string, lang='gu'):
#   tokenized_sentences.append(i)
# (tokenized_sentences)
# 8058 8046

Streaming output truncated to the last 5000 lines.
6397 44
6398 48
6399 54
6400 59
6401 26
6402 43
6403 56
6404 16
6405 63
6406 26
6407 19
6408 33
6409 23
6410 42
6411 16
6412 36
6413 58
6414 46
6415 31
6416 17
6417 30
6418 58
6419 49
6420 13
6421 53
6422 30
6423 41
6424 55
6425 55
6426 27
6427 23
6428 38
6429 41
6430 50
6431 51
6432 40
6433 13
6434 34
6435 52
6436 37
6437 18
6438 27
6439 40
6440 29
6441 51
6442 15
6443 18
6444 52
6445 47
6446 16
6447 51
6448 30
6449 23
6450 19
6451 60
6452 7
6453 18
6454 54
6455 42
6456 6
6457 25
6458 29
6459 33
6460 36
6461 20
6462 26
6463 24
6464 4
6465 43
6466 17
6467 42
6468 41
6469 47
6470 33
6471 30
6472 26
6473 24
6474 14
6475 37
6476 19
6477 22
6478 31
6479 37
6480 22
6481 54
6482 23
6483 33
6484 26
6485 30
6486 16
6487 53
6488 28
6489 30
6490 41
6491 18
6492 25
6493 32
6494 45
6495 27
6496 44
6497 40
6498 29
6499 24
6500 37
6501 50
6502 29
6503 69
6504 23
6505 34
6506 49
6507 39
6508 35
6509 33
6510 56
6511 6
6512 19
6513 30
6514 49
6515 39
6

AttributeError: ignored

In [ ]:
indic_string=train_data['text'][6412]
tokenized_sentences=[]
for i in sentence_tokenize.sentence_split(indic_string, lang='gu'):
  tokenized_sentences.append(i)
(indic_string)

'પરિવારે આરોપ મૂકયો તેમના પરિવારની મહિલાને પ્રસૂતિકાળમાં સારવાર અર્થે હૉસ્પિટલ લઈ જવામાં આવી રહી હતી તે દરમિયાન પોલીસે ગાડીને રોકી હતી અને તેનાં કારણે મોડું થતાં ગર્ભમાં જ બાળકનું મૃત્યુ થયું. જે મહિલાની બાળકીનું ગર્ભમાં મૃત્યુ થયું તેમનાં દિયર મોતીભાઈએ બીબીસી ગુજરાતી સાથેની વાતચીતમાં કહ્યું કે, "પોલીસ પ્રસૂવ પીડિત મહિલાને સારવાર માટે લઈ જતી ગાડીને કેવી રીતે રોકી શકે?" આ ઘટનામાં પોલીસે ફરિયાદ લીધી છે તપાસ થઈ રહી છે. શું હતો ઘટનાક્રમ? પરિવાર પોલીસ સ્ટેશને મોતીભાઈ દિવસના ઘટનાક્રમ વિશે વાત કરતા કહે છે, રવિવારે મોડી રાત્રે મારા ભાભીને પ્રસવની પીડા થતા અમે તેમને અંબાજી જનરલ હૉસ્પિટલમાં લઈ ગયા હતા. ત્યાંનાં ડૉક્ટરોએ કહ્યું કે મહિલાની સ્થિતિ ગંભીર હોવાથી તેમને પાલનપુર લઈ જવામાં આવે. મોહિતભાઈ પોતે ડ્રાઇવિંગનું કામ કરે છે અને પોતે જ એમના ભાભીને ગાડીમાં હૉસ્પિટલ લઈ ગયા હતા. તેઓ કહે છે, "અમે જ્યારે મારા ભાભીને પાલનપુર લઈ જઈ રહ્યા હતા ત્યારે અમને પોલીસ દ્વારા ડી.કે.સર્કલ પાસે ઊભા રાખવામાં આવ્યાં." "પોલીસ સાથેની વાતચીત દરમિયાન મેં મોંઢે માસ્કની જગ્યાએ રૂમાલ બાંધેલો હતો જે છૂટી ગયો. અમે એટલી ઉતાવળમા

In [ ]:
stop_words=[]
stop_word_txt='/content/drive/MyDrive/Colab Notebooks/A.List.of.210.Gujarati.Stop.Words.txt'
for line in open(stop_word_txt,'r'):
  without_spaces=line.rstrip()
  word=without_spaces.split('\n')
  stop_words.append(word)
stop_words=stop_words[:210]
stop_words[0]='અથવા'
stop=['અથવા']
for i in stop_words[1:]:
  stop.append(i[0])
stop_words=stop

In [ ]:
def cleanText(text):
    # text=re.sub(r'(\d+)',r'',text)
    text=text.replace(u',','')
    text=text.replace(u'"','')
    text=text.replace(u'"','')
    text=text.replace(u':','')
    text=text.replace(u"'",'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"?",'')
    text=text.replace(u")",'')
    text=text.replace(u"(",'')
    return text

In [ ]:
# remove punctuations
sentences_clean=[cleanText(sentence) for sentence in tokenized_sentences]

In [ ]:
no_stop_sent=[]
for sentence in sentences_clean:
  for item in [x for x in sentence.split(' ') if not str(x) in stop_words]:
    no_stop_sent.append(item)

In [ ]:
# stop word elimination
no_stop_sentence=[]
no_stop_sentences=[]
list_no_stop_sentences=[]
for sentence in sentences_clean:
  for item in [x for x in sentence.split(' ') if not str(x) in stop_words]:
    no_stop_sentence.append(item)
  # no_stop_sentences.append(no_stop_sentence)
  list_no_stop_sentence=" ".join(no_stop_sentence)
  no_stop_sentences.append(no_stop_sentence)
  list_no_stop_sentences.append(list_no_stop_sentence)
  list_no_stop_sentence=[]
  no_stop_sentence=[]
len(no_stop_sentences)

36

In [ ]:
# word embedding
from gensim.models import Word2Vec
w2v=Word2Vec(no_stop_sentences,size=1,min_count=1,iter=1000)
# print(w2v)
sentence_embeddings=[[w2v[word][0] for word in words] for words in no_stop_sentences]
# len(sentence_embeddings)
max_len=max([len(tokens) for tokens in no_stop_sentences])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
len(sentence_embeddings)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


36

In [ ]:
# similarity matrix
from scipy import spatial
import networkx as nx
similarity_matrix = np.zeros([len(no_stop_sentences), len(no_stop_sentences)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
# print(w2v['કરવામાં'])

In [ ]:
# page ranking
import math
from collections import OrderedDict
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)
# print(scores)
top_sentence={sentence:scores[index] for index,sentence in enumerate(tokenized_sentences)}
# Printing sorted dictionary
print("Sorted dictionary using sorted() and keys() is : ")
top=dict(sorted(top_sentence.items(), key=lambda x: x[0], reverse=True)[:math.floor(len(tokenized_sentences)*0.4)])
# sorted_dict = sorted(top)

Sorted dictionary using sorted() and keys() is : 


In [ ]:
counter=0
sumamry_w2v=""
for sent in tokenized_sentences:
    if sent in top.keys():
      sumamry_w2v+=sent
      print(sent)
      counter+=1
print(sumamry_w2v)

પરિવારે આરોપ મૂકયો તેમના પરિવારની મહિલાને પ્રસૂતિકાળમાં સારવાર અર્થે હૉસ્પિટલ લઈ જવામાં આવી રહી હતી તે દરમિયાન પોલીસે ગાડીને રોકી હતી અને તેનાં કારણે મોડું થતાં ગર્ભમાં જ બાળકનું મૃત્યુ થયું.
શું હતો ઘટનાક્રમ?
પરિવાર પોલીસ સ્ટેશને મોતીભાઈ દિવસના ઘટનાક્રમ વિશે વાત કરતા કહે છે, રવિવારે મોડી રાત્રે મારા ભાભીને પ્રસવની પીડા થતા અમે તેમને અંબાજી જનરલ હૉસ્પિટલમાં લઈ ગયા હતા.
ત્યાંનાં ડૉક્ટરોએ કહ્યું કે મહિલાની સ્થિતિ ગંભીર હોવાથી તેમને પાલનપુર લઈ જવામાં આવે.
મોહિતભાઈ પોતે ડ્રાઇવિંગનું કામ કરે છે અને પોતે જ એમના ભાભીને ગાડીમાં હૉસ્પિટલ લઈ ગયા હતા.
તેઓ કહે છે, "અમે જ્યારે મારા ભાભીને પાલનપુર લઈ જઈ રહ્યા હતા ત્યારે અમને પોલીસ દ્વારા ડી. કે. સર્કલ પાસે ઊભા રાખવામાં આવ્યાં.
માસ્ક લેવાનું ભૂલી ગયા હતા માટે રૂમાલ બાંધ્યો હતો.
રૂમાલ છૂટતાં પોલીસે અમારી પાસેથી 200 રૂપિયા દંડ માગ્યો હતો.
પાલનપુરમાં લોહીનો પ્રશ્ન અંબાજીથી એક કલાકથી વધારે સમયની મુસાફરી કરીને પરિવાર પ્રસવપીડિત મહિલાને લઈને પાલનપુર પહોંચ્યો.
મોતીભાઈ કહે છે, "અમે જ્યારે પાલનપુર પહોંચ્યા ત્યારે ત્યાંના ડૉક્ટરોએ સોનોગ્રાફી કરી.
સિઝરિયન કરીને

In [ ]:
# !pip install fasttext
# import fasttext
# import fasttext.util
# fasttext.util.download_model('gu', if_exists='ignore')

In [ ]:
# ft=fasttext.load_model('/content/drive/MyDrive/Colab Notebooks/cc.gu.300.bin')

In [ ]:
# vec=[]
# for i in range(0,len(list_no_stop_sentences)):
#     vec.append(ft.get_sentence_vector(list_no_stop_sentences[i]))
# [w2v[word][0] for word in words] for words in no_stop_sentences]

244

iNLTK

In [ ]:
# !pip install inltk
# from inltk.inltk import get_embedding_vectors
# !pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
# from inltk.inltk import setup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.8 MB/s 
     |████████████████████████████████| 355 kB 59.4 MB/s 
     |████████████████████████████████| 233 kB 67.8 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19190 sha256=db42ed60d7622b419c7b9a2a22424281503aa77266a9d51987589a241bcb5367
  Stored in directory: /root/.cache/pip/wheels/df/99/da/c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26325 sha256=6c141c67783d5e80c2454431502e89d67126e3d39375a49ca496fc39ad489288
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built nvidia-ml-py3 typing
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.9
    Uninstalling fastai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 111.8 MB 96 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.3.1+cpu which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.3.1+cpu which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.3.1+cpu which is incompatible.


In [ ]:
 from inltk.inltk import setup
 setup('gu')

RuntimeError: ignored

We'll only do this for the first time.


In [ ]:
counter=0
import warnings
similarity_matrix_inltk=np.zeros([len(no_stop_sentences), len(no_stop_sentences)])
warnings.filterwarnings("ignore")
from inltk.inltk import get_sentence_similarity
dim=int(len(no_stop_sentences))
similarity_matrix=np.zeros((dim,dim))
for i in range(0,math.floor(len(tokenized_sentences))):
  similarity_matrix[i][i]=1
  for j in range(i,len(tokenized_sentences)):
    similarity_matrix_inltk[i][j]=get_sentence_similarity(list_no_stop_sentences[i],list_no_stop_sentences[j],'gu')
    similarity_matrix_inltk[j][i]=similarity_matrix_inltk[i][j]
  counter+=1
  print(counter)

1


2


3


4


5


...

32


33


34


35


36


In [ ]:
similarity_matrix_inltk

array([[ 1.      ,  0.08089 ,  0.059393,  0.088603, ...,  0.143997,  0.15685 ,  0.061924, -0.005575],
       [ 0.08089 ,  1.      ,  0.207464,  0.043784, ...,  0.037889,  0.285481,  0.352946,  0.031099],
       [ 0.059393,  0.207464,  1.      ,  0.281461, ...,  0.085438,  0.166669,  0.16956 ,  0.02078 ],
       [ 0.088603,  0.043784,  0.281461,  1.      , ...,  0.050624,  0.140912, -0.019346,  0.026975],
       ...,
       [ 0.143997,  0.037889,  0.085438,  0.050624, ...,  1.      , -0.014844, -0.032429,  0.038972],
       [ 0.15685 ,  0.285481,  0.166669,  0.140912, ..., -0.014844,  1.      ,  0.165758,  0.123454],
       [ 0.061924,  0.352946,  0.16956 , -0.019346, ..., -0.032429,  0.165758,  1.      ,  0.13778 ],
       [-0.005575,  0.031099,  0.02078 ,  0.026975, ...,  0.038972,  0.123454,  0.13778 ,  1.      ]])

In [ ]:
import math
import networkx as nx
nx_graph = nx.from_numpy_array(similarity_matrix_inltk)
scores = nx.pagerank(nx_graph)
print(scores)
top_sentence={sentence:scores[index] for index,sentence in enumerate(tokenized_sentences)}
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:math.floor(len(tokenized_sentences)*0.4)])

{0: 0.023312796413071366, 1: 0.030296400025691846, 2: 0.022084474135393814, 3: 0.01857562774943889, 4: 0.030729713245693208, 5: 0.022655704903293022, 6: 0.028607653612265853, 7: 0.03246070719627751, 8: 0.02191161184971155, 9: 0.029325382256251496, 10: 0.03667075054658844, 11: 0.035253358020933975, 12: 0.03268898916303576, 13: 0.035028666338423865, 14: 0.02921713785645523, 15: 0.031858503204123735, 16: 0.022823479631840712, 17: 0.03384323498224493, 18: 0.014421339302516763, 19: 0.029035034582520255, 20: 0.030256660044847984, 21: 0.03187295457089663, 22: 0.021929905528839534, 23: 0.027053087335881784, 24: 0.029085432202875003, 25: 0.019251106847652966, 26: 0.030868184572787664, 27: 0.027652297071793917, 28: 0.0333287081021524, 29: 0.0231101702171164, 30: 0.038117698359866566, 31: 0.029265513107322748, 32: 0.01582819708886385, 33: 0.03620754601177609, 34: 0.025759269115622735, 35: 0.019612704805931518}


In [ ]:
counter=0
summary_inltk=""
for sent in tokenized_sentences:
    if sent in top.keys():
      summary_inltk+=sent
      print(sent)
        # counter+=1
print(summary_inltk)

જે મહિલાની બાળકીનું ગર્ભમાં મૃત્યુ થયું તેમનાં દિયર મોતીભાઈએ બીબીસી ગુજરાતી સાથેની વાતચીતમાં કહ્યું કે, "પોલીસ પ્રસૂવ પીડિત મહિલાને સારવાર માટે લઈ જતી ગાડીને કેવી રીતે રોકી શકે?
પરિવાર પોલીસ સ્ટેશને મોતીભાઈ દિવસના ઘટનાક્રમ વિશે વાત કરતા કહે છે, રવિવારે મોડી રાત્રે મારા ભાભીને પ્રસવની પીડા થતા અમે તેમને અંબાજી જનરલ હૉસ્પિટલમાં લઈ ગયા હતા.
તેઓ કહે છે, "અમે જ્યારે મારા ભાભીને પાલનપુર લઈ જઈ રહ્યા હતા ત્યારે અમને પોલીસ દ્વારા ડી. કે. સર્કલ પાસે ઊભા રાખવામાં આવ્યાં.
માસ્ક લેવાનું ભૂલી ગયા હતા માટે રૂમાલ બાંધ્યો હતો.
રૂમાલ છૂટતાં પોલીસે અમારી પાસેથી 200 રૂપિયા દંડ માગ્યો હતો.
તે થોડી રકઝક બાદ અમે આપી દીધો હતો.
" પોલીસ સ્ટેશને સગર્ભાને ગાડીમાંથી બહાર ઉતાર્યાં પરિવારનાં બહેન પોલીસ સ્ટેશનમાં મોતીભાઈ જણાવે છે કે દંડ ચૂકવ્યા પછી "પોલીસ અધિકારીઓ ગાડી ડિટેઇન કરીને પોલીસ સ્ટેશન લઈ ગયા હતા, જ્યાં સગર્ભા સહિત પરિવારને ગાડીની બહાર ઉતારવામાં આવ્યો હતો.
" મોતીભાઈ કહે છે, "અમે તેમને વિનંતીઓ કરી કે મહિલાને પ્રસવપીડા છે અને ખૂબ જ તકલીફ થઈ રહી છે છતાં તેમણે અમારી વાત ન માની.
પાલનપુરમાં લોહીનો પ્રશ્ન અંબાજીથી 

In [ ]:
!pip uninstall torch
!pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
setup('gu')

Found existing installation: torch 1.3.1+cpu
Uninstalling torch-1.3.1+cpu:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.3.1+cpu.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.3.1+cpu



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cpu/torch-1.3.1%2Bcpu-cp37-cp37m-linux_x86_64.whl (111.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.3.1+cpu which is incompatible.
torchtext 0.13.1 requires torch==1.

RuntimeError: ignored

In [ ]:
for i in (tokenized_sentences):
  print(i)

નરેન્દ્ર દામોદરદાસ મોદી (જન્મ: ૧૭ સપ્ટેમ્બર ૧૯૫૦)[૨] ભારતના ૧૪મા વડાપ્રધાન છે.
તેઓ ગુજરાત રાજ્યના મુખ્ય મંત્રી પણ રહી ચૂક્યા છે અને ભારતીય જનતા પાર્ટી (ભાજપ)ના નેતા છે.
નરેન્દ્ર દામોદરદાસ મોદીનો જન્મ એક મધ્યમ વર્ગના હિન્દુ કુટુંબમાં અમદાવાદથી ૧૧૨ કીમી અને મહેસાણાથી ૩૪ કીમી દૂર વડનગર ખાતે થયો હતો.
તે દામોદરદાસ મૂલચંદ મોદી અને તેમના પત્ની હીરાબેન મોદીના છ સંતાન પૈકી ત્રીજુ સંતાન છે.
[૩] તેમનું હુલામણું નામ એન. ડી. હતું. તે નાનપણથી જ રાષ્ટ્રીય સ્વયંસેવક સંઘના સભ્ય હતા.
તેઓ ધાર્મિક હતા અને મહાદેવના મંદિરે જતા હતા.
તેઓએ ૧૭ વર્ષની વયે ગૃહ ત્યાગ કર્યો હતો.
તે હિમાલયમાં તેમજ રામકૃષ્ણ પરમહંસના બેલુર મઠમાં પણ રહી ચૂક્યા છે.
ઈંદિરા ગાંધીએ લાદેલી કટોકટી વખતે તેઓ ભૂગર્ભમાં જતા રહ્યા હતા.
આ કટોકટી વખતે જ તેમણે સંઘર્ષમાં ગુજરાત નામનું પુસ્તક લખ્યું હતું.
તેમને કિશોરાવસ્થાથી રાજકારણમાં રસ હતો.
તેમણે ગુજરાત યુનિવર્સિટીમાંથી રાજકીય વિજ્ઞાનમાં અનુસ્નાતકની પદવી પ્રાપ્ત કરી છે.
૧૯૯૮માં ભારતીય જનતા પાર્ટી (ભાજપ)ના નેતા લાલકૃષ્ણ અડવાણીએ ગુજરાત અને હિમાચલ પ્રદેશની ચૂંટણીમાં પ્રચાર માટે તેમની પસંદગી કરી હતી. ઇ

In [ ]:
string="મુવીઝ હોઈ કે પકશી એન્સિયન્ટ આર્ટ આપણે તે બધા ની અંદર જોઈએ છીએ કે ઘણા બધા લોકો આનંદ માટે લાંબા પાઇપ પકડી ને બેઠા હોઈ છે અને તેની અંદર થી ધુમાડો કાઢતા હોઈ છે. અને સિગરેટ સ્મોકિંગ અથવા હૂકા સ્મોકિંગ એ એક એવી વસ્તુ છે કે જે આટલા બધા વરસો બાદ પણ ખુબ જ પ્રખ્યાત છે અને આજે પણ લોકો સ્મોક કરે છે. અને આટલા વરસો ની અંદર હૂકા ની અંદર ઘણા બધા ફેરફાર કરવા માં આવ્યા છે પરંતુ આજે પણ હૂકા ઘણા બધા લોકો ની મન્પસંદન વસ્તુ છે. સામાન્ય રીતે હૂકા એ એક વોટર પાઇપ છે જે ટોબેકો સ્મોક કરવા માટે વાપરવા માં આવે છે. આ પાઇપ લમ્બો હોઈ છે અને વોટર ચેમ્બર સાથે જોડાયેલ હોઈ છે. અને તેની અંદર ઘણી વખત એક અથવા વધુ પાઇપ જોડેવા માં આવેલ છે જે એક કરતા વધુ સ્મોકર્સ માટે બનાવવા માં આવેલ હોઈ છે.હુક્કા સ્મોકિંગ સાથે સંકળાયેલ હેલ્થ રિસ્ક.અને સામાન્ય રીતે હૂકા ની અંદર જે ટોબેકો નો ઉપીયોગ કરવા માં આવે છે તેની અંદર ફ્રૂટ ની ફ્લેવર મિક્સ કરી અને તેને ગાળ્યો બનાવવા માં આવે છે અને તેના માટે તેની અંદર કોકોનટ, ફ્રૂટ ફ્લેવર, મિન્ટ અથવા કોફી નો ઉપીયોગ કરવા માં આવે છે. અને હૂકા ની અંદર આ જે ફ્લેવર ને એડ કરવા માં આવે છે તેના કારણે જ તે યુવાનો ની અંદર ખુબ જ પ્રખ્યાત છે. અને આ સ્મોક કરવા થી જે લોકો ને મજા આવે છે તેની સામે તેઓ તેના દ્વારા તમારા શરીર ને જે નુકસાન પહોંચાડે છે તેને નજરઅંદાજ કરે છે.હૂકા લોકો 400 કરતા પણ વધુ વર્ષો થી પિતા આવ્યા છે, અને હૂકા નો આવિશકર એક મિસ બ્લિલીફ સાથે કરવા માં આવ્યો હતો કે જે ટોબેકો ને સ્મોક કરવા માં આવે છે જો તેને પાણી દ્વારા કરવા માં આવે તો તેના દ્વારા જે શરીર ને નુકસાન પહોંચે છે તે ઓછું થશે. તો આવો આપણે જાણીયે કે કઈ રીતે હુક્કા પીવા થી તે આપણા શરીર ને કેટલું નુકસાન પહોંચાડે છે."

In [ ]:
for i in sentence_tokenize.sentence_split(string, lang='gu'):
  tokenized_sentences.append(i)
len(tokenized_sentences)

12